In [68]:
import pandas as pd
import matplotlib.pyplot as plt
from enum import Enum
import numpy as np

df = pd.read_excel('survey-results.xlsx')
TOTAL_RESPONDENTS = len(df)

print(df.columns)

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q7', 'Q6', 'Q8', 'Q9', 'Q10',
       'Q1_1', 'Q1_2', 'Q1_3', 'Q1_4', 'Q1_5', 'Q1_6', 'Q1_7', 'Q1_8', 'Q1_9',
       'Q1_10', 'Q3_1', 'Q3_2', 'Q3_3', 'Q3_4', 'Q3_5', 'Q3_6', 'Q5_1', 'Q5_2',
       'Q5_3', 'Q5_4', 'Q5_5', 'Q5_6', 'Q5_7', 'Q5_8', 'Q5_9', 'Q5_10',
       'Q5_11', 'Q5_12', 'Q5_13', 'Q5_14', 'Q5_15'],
      dtype='object')


In [69]:
ages_count = df['Q8'].value_counts().sort_index(ascending=True)
not_answered_df = pd.Series([TOTAL_RESPONDENTS - ages_count.sum()], index=['Not Answered'])
ages_count = pd.concat([ages_count, not_answered_df])

gender_count = df['Q9'].value_counts().sort_index(ascending=True)
not_answered_df = pd.Series([TOTAL_RESPONDENTS - gender_count.sum()], index=['Not Answered'])
gender_count = pd.concat([gender_count, not_answered_df])

injury_count = df['Q10'].value_counts().sort_index(ascending=True)
not_answered_df = pd.Series([TOTAL_RESPONDENTS - injury_count.sum()], index=['Not Answered'])
injury_count = pd.concat([injury_count, not_answered_df])

print(ages_count)
print(gender_count)
print(injury_count)

16 - 24                  2
Please enter your age    1
Not Answered             0
dtype: int64
Female                      1
Male                        1
Please enter your gender    1
Not Answered                0
dtype: int64
Have you ever been injured whilst cycling?    1
Yes - major injury                            1
Yes - minor injury                            1
Not Answered                                  0
dtype: int64


In [71]:
ratings = ["Strongly disagree", "Somewhat disagree", "Neutral", "Somewhat agree", "Strongly agree"]
sus_scores = {}
sus_totals = []

for p in range(1, TOTAL_RESPONDENTS):
    sus_scores[p] = []
            
    # get sus scores
    for q in range(1, 11):
        que = 'Q1_' + str(q)
        rating = df[que][p]
        score = ratings.index(rating)
        if q % 2 == 0: 
            score = 5 - (score+1)
        sus_scores[p].append(score)
    sus_totals.append(sum(sus_scores[p])*2.5)
    
    # get nasa-tlx scores
    for q in range(1, 7):
        que = 'Q3_' + str(q)
        weight = (df[que][p]) * 5
        print(weight)


print(f"Sus totals: {sus_totals} with an average sus score of {np.mean(sus_totals):.2f}")       

90
45
60
25
45
100
70
35
90
55
50
55
Sus totals: [45.0, 52.5] with an average sus score of 48.75
